In [40]:
# Install required libraries
!pip install gensim openpyxl nltk rouge-score bert_score tqdm --quiet

# Download necessary NLTK data
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [41]:
from preprocessingword2vec import load_and_preprocess_data

# ✅ File paths
train_path = "/content/MeQSum_ACL2019_BenAbacha_Demner-Fushman.xlsx"
val_path   = "/content/MEDIQA2021-Task1-QuestionSummarization-ValidationSet.xlsx"
test_path  = "/content/MEDIQA2021-Task1-TestSet-ReferenceSummaries.xlsx"
w2v_path   = "/content/w2v_frozen.model"  # ensure this is uploaded

# ✅ Load and preprocess data
data = load_and_preprocess_data(
    train_path=train_path,
    val_path=val_path,
    test_path=test_path,
    w2v_path=w2v_path,      # match the function signature
    max_len_q=50,
    max_len_s=20,
    embedding_dim=128
)

# ✅ Unpack processed data
X_train          = data["X_train"]
y_train          = data["y_train"]
X_val            = data["X_val"]
y_val            = data["y_val"]
X_test           = data["X_test"]
y_test           = data["y_test"]
tokenizer        = data["tokenizer"]
vocab_size       = data["vocab_size"]
max_len_q        = data["max_len_q"]
max_len_s        = data["max_len_s"]
embedding_matrix = data["embedding_matrix"]


🔹 Step 1: Loading datasets...
   ↪ df_train shape: (1000, 3)
   ↪ df_val shape: (50, 5)
   ↪ df_test shape: (100, 3)
🔹 Step 2: Cleaning text and adding SOS/EOS tokens...
Sample clean_summary with SOS/EOS: sos who manufactures cetirizine? eos
Tokenized summary: ['sos', 'who', 'manufactures', 'cetirizine', '?', 'eos']
Sample clean_summary with SOS/EOS: sos should i leave a meniscus tear untreated if it does not cause problems? eos
Tokenized summary: ['sos', 'should', 'i', 'leave', 'a', 'meniscus', 'tear', 'untreated', 'if', 'it', 'does', 'not', 'cause', 'problems', '?', 'eos']
Sample clean_summary with SOS/EOS: sos how can i get rid of a lower lip birthmark permanently? eos
Tokenized summary: ['sos', 'how', 'can', 'i', 'get', 'rid', 'of', 'a', 'lower', 'lip', 'birthmark', 'permanently', '?', 'eos']
   ↪ Sample cleaned question: subject who and where to get cetirizine d message i needwant to know who manufscturs cetirizine my walmart is looking for a new supply and are not getting the rec

In [42]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Dot, Activation, TimeDistributed
import tensorflow.keras.backend as K

latent_dim = 256
embedding_dim = 128  # must match your embedding_matrix dimension

# ----- ENCODER -----
encoder_inputs = Input(shape=(max_len_q,), name="encoder_input")
enc_emb = Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    trainable=False,
    mask_zero=False         # disable masking to avoid BroadcastTo errors
)(encoder_inputs)

encoder_lstm = LSTM(
    latent_dim,
    return_sequences=True,
    return_state=True,
    dropout=0.3,
    recurrent_dropout=0.3,
    name="encoder_lstm"
)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# ----- DECODER -----
decoder_inputs = Input(shape=(max_len_s - 1,), name="decoder_input")
dec_emb = Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    trainable=False,
    mask_zero=False         # also disable masking here
)(decoder_inputs)

decoder_lstm = LSTM(
    latent_dim,
    return_sequences=True,
    return_state=True,
    dropout=0.3,            # disable CuDNN path
    recurrent_dropout=0.3,  # disable CuDNN path
    implementation=1,       # force generic TF LSTM
    name="decoder_lstm"
)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# ----- ATTENTION -----
score = Dot(axes=[2, 2], name="attention_score")([decoder_outputs, encoder_outputs])
attention_weights = Activation('softmax', name="attention_weights")(score)
context_vector = Dot(axes=[2, 1], name="context_vector")([attention_weights, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1, name="decoder_concat_context")(
    [context_vector, decoder_outputs]
)

# 🔁 Optional Dense Layer for richer representations
intermediate = TimeDistributed(Dense(256, activation='tanh'))(decoder_combined_context)
output = Dense(vocab_size, activation='softmax', name="final_output")(intermediate)

# 🔧 Compile Model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

# 🎯 Attention Model for visualization
attention_model = Model(
    [encoder_inputs, decoder_inputs],
    [output, attention_weights]
)


Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_22        │ (None, 50, 128)   │  1,052,416 │ encoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_23        │ (None, 19, 128)   │  1,052,416 │ decoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 50, 256), │    394,240 │ embedding_22[0][… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, 19, 256), │    394,240 │ embedding_23[0][… │
│                     │ (None, 256),      │            │ encoder_lstm[0][… │
│                     │ (None, 256)]      │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_score     │ (None, 19, 50)    │          0 │ decoder_lstm[0][… │
│ (Dot)               │                   │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_weights   │ (None, 19, 50)    │          0 │ attention_score[… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ context_vector      │ (None, 19, 256)   │          0 │ attention_weight… │
│ (Dot)               │                   │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_concat_con… │ (None, 19, 512)   │          0 │ context_vector[0… │
│ (Concatenate)       │                   │            │ decoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_11 │ (None, 19, 256)   │    131,328 │ decoder_concat_c… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_output        │ (None, 19, 8222)  │  2,113,054 │ time_distributed… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,137,694 (19.60 MB)

 Trainable params: 3,032,862 (11.57 MB)

 Non-trainable params: 2,104,832 (8.03 MB)

In [43]:
# Correct reverse lookup and token IDs
reverse_tokenizer = {v: k for k, v in tokenizer.word_index.items()}
sos_token = tokenizer.word_index.get('sos', tokenizer.word_index.get('SOS'))
eos_token = tokenizer.word_index.get('eos', tokenizer.word_index.get('EOS'))

def decode_sequence(input_seq):
    # Prepare decoder input of integer token IDs
    decoder_input = np.zeros((1, max_len_s - 1), dtype=int)
    decoder_input[0, 0] = sos_token

    output_seq = []
    for i in range(max_len_s - 1):
        # Predict next token probabilities
        preds = model.predict([input_seq, decoder_input], verbose=0)
        pred_id = np.argmax(preds[0, i])
        word = reverse_tokenizer.get(pred_id, '')

        if word == 'eos':
            break
        if word.lower() not in ('sos', 'eos', '<OOV>', ''):
            output_seq.append(word)

        # Feed predicted ID into next timestep
        decoder_input[0, i] = pred_id

    return ' '.join(output_seq)


In [44]:
import numpy as np
import math
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# Correct reverse lookup and token IDs
reverse_tokenizer = {v: k for k, v in tokenizer.word_index.items()}
sos_token = tokenizer.word_index.get('sos', tokenizer.word_index.get('SOS'))
eos_token = tokenizer.word_index.get('eos', tokenizer.word_index.get('EOS'))

def decode_sequence_beam(input_seq, beam_width=5):
    # Each candidate: (token list, score, decoder_input array)
    sequences = [[[], 0.0, np.zeros((1, max_len_s - 1), dtype=int)]]
    sequences[0][2][0, 0] = sos_token

    for _ in range(max_len_s - 2):
        all_candidates = []
        for seq, score, decoder_input in sequences:
            preds = model.predict([input_seq, decoder_input], verbose=0)
            prob_dist = preds[0, len(seq)]
            top_ids = np.argsort(prob_dist)[-beam_width:]

            for tok_id in top_ids:
                word = reverse_tokenizer.get(tok_id, '')
                if word.lower() in ('sos', '<OOV>', ''):
                    continue
                new_seq = seq + [word]
                # length penalty
                new_score = (score - np.log(prob_dist[tok_id] + 1e-10)) / (len(seq) + 1)
                new_decoder_input = decoder_input.copy()
                if len(seq) + 1 < max_len_s - 1:
                    new_decoder_input[0, len(seq) + 1] = tok_id
                all_candidates.append([new_seq, new_score, new_decoder_input])

        # keep best beams
        sequences = sorted(all_candidates, key=lambda tup: tup[1])[:beam_width]

        # stop if all sequences have ended
        if all('eos' in seq for seq, _, _ in sequences):
            break

    # choose best sequence and strip eos if present
    final_seq = sequences[0][0]
    if 'eos' in final_seq:
        final_seq = final_seq[:final_seq.index('eos')]
    return ' '.join(w for w in final_seq if w.lower() not in ('sos', '<OOV>'))

# 📊 Evaluation Setup
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
smoothie = SmoothingFunction().method4
refs, preds = [], []

# 🧠 Perplexity
val_loss = model.evaluate([X_val, decoder_input_val], decoder_target_val, verbose=0)[0]
perplexity = math.exp(val_loss)
print(f"🧠 Validation Perplexity: {perplexity:.2f}\n")

# 🔍 Sample Evaluation
for idx in range(10):
    input_seq = X_val[idx:idx+1]
    ref_ids   = y_val[idx]

    reference = [
        reverse_tokenizer.get(tok, '')
        for tok in ref_ids
        if tok not in (0, sos_token, eos_token)
    ]
    reference = [w for w in reference if w]

    prediction = decode_sequence_beam(input_seq, beam_width=5)

# 📝 BERTScore
print("\nBERTScore Results:")
print(f"Precision : {abs(P.mean().item()):.4f}")
print(f"Recall    : {abs(R.mean().item()):.4f}")
print(f"F1 Score  : {abs(F1.mean().item()):.4f}")


🧠 Validation Perplexity: 8295.27


BERTScore Results:
Precision : 0.5212
Recall    : 0.2301
F1 Score  : 0.3785
